# Модель (*)ARCH (пакет [`arch`](https://arch.readthedocs.io/en/latest/index.html))

In [ ]:
import numpy as np
import pandas as pd

from arch import arch_model

# Модель среднего
from arch.univariate import ARX, ARCHInMean 
# Модель волатильности
from arch.univariate import GARCH, FIGARCH, EGARCH, APARCH, EWMAVariance
# Распределение для epsilon
from arch.univariate import Normal, StudentsT, SkewStudent

import pandas_datareader.data as web

# настройки визуализации
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)

Основные модели (*)ARCH
* [`ARCH(p)`](https://arch.readthedocs.io/en/latest/univariate/generated/arch.univariate.ARCH.html#arch.univariate.ARCH)
* [`GARCH(p,0,q,power)`](https://arch.readthedocs.io/en/latest/univariate/generated/arch.univariate.GARCH.html#arch.univariate.GARCH)
* [`EGARCH(p,q)`](https://arch.readthedocs.io/en/latest/univariate/generated/arch.univariate.EGARCH.html#arch.univariate.EGARCH)
* [`APARCH(p,o,q,delta)`](https://arch.readthedocs.io/en/latest/univariate/generated/arch.univariate.APARCH.html#arch.univariate.APARCH)
* [`EWMAVariance(lambda)`](https://arch.readthedocs.io/en/latest/univariate/generated/arch.univariate.EWMAVariance.html#arch.univariate.EWMAVariance)

Для модели `GARCH-in-mean` используем класс [`ARCHInMean`](https://arch.readthedocs.io/en/latest/univariate/generated/arch.univariate.ARCHInMean.html#arch.univariate.ARCHInMean)

Загрузим из БД [`FRED`](https://fred.stlouisfed.org/) недельные данные по 3-Month Treasury Bill Secondary Market Rate (Symbol [`WTB3MS`](https://fred.stlouisfed.org/series/WTB3MS)) с 2000-01-01 по 2023-12-31 и создадим датафрейм `y` - первая разность ставки

In [ ]:
rate = web.DataReader(name='WTB3MS', data_source='fred', start='2000-01-01', end='2023-12-31')
y = rate.diff().dropna()

y.plot()
plt.show()

## Подгонка AR-(*)ARCH заданного порядка  с использованием `arch_model`


Подгоним модель AR(2)-GARCH(1,1)

Спецификация

\begin{align*}
	y_t&=\alpha_0+\phi_1y_{t-1}+\phi_2y_{t-2}+u_t \\ u_t&=\sigma_t \epsilon_t \\
	\sigma^\lambda_t &=\omega+\alpha|\epsilon_{t-1}|^\lambda+\beta\sigma^\lambda_{t-1}\\
	\epsilon_t&\sim N(0,1)
\end{align*}

где $\lambda=2$

In [ ]:
am = arch_model(y, mean='ARX', lags=2, vol='GARCH', p=1, q=1, power=2, dist='normal')

res = am.fit(disp="off")

res.summary()

In [ ]:
res.plot(annualize='W')
plt.show()

In [ ]:
res.hedgehog_plot(plot_type='volatility')
plt.show()

In [ ]:
res.hedgehog_plot(plot_type='mean')
plt.show()

In [ ]:
res.arch_lm_test(lags=5)

## Подгонка AR-(*)ARCH заданного порядка

Подгоним модель AR(1)-GARCH(1,1,1)

Спецификация

\begin{align*}
	y_t&=\alpha_0+\phi_1y_{t-1}+u_t \\ u_t&=\sigma_t \epsilon_t \\
	\sigma^\lambda_t &=\omega+\alpha|\epsilon_{t-1}|^\lambda+\gamma|\epsilon_{t-1}|^\lambda I(\epsilon_{t-1}<0)
	 +\beta\sigma^\lambda_{t-1}\\
	 \epsilon_t&\sim t
\end{align*}

где $\lambda=2$

In [ ]:
ar = ARX(y, lags=1)
ar.volatility = GARCH(p=1, o=1, q=1, power=2)
ar.distribution = StudentsT(seed=2)

res = ar.fit(disp="off")

res.summary()

## Прогнозирование

построим прогноз на 5 периодов по модели AR(1)-GARCH(1,1)

Спецификация

\begin{align*}
	y_t&=\alpha_0+\phi y_{t-1}+u_t \\ u_t&=\sigma_t \epsilon_t \\
	\sigma^\lambda_t &=\omega+\alpha |\epsilon_{t-1}|^\lambda+\beta\sigma^\lambda_{t-1}\\
	\epsilon_t&\sim t
\end{align*}

где $\lambda=2$

In [ ]:
am = arch_model(y, mean='ARX', lags=1, vol='GARCH', p=1, q=1, o=0, dist='t')
res = am.fit(disp="off")

y_forecasts = res.forecast(horizon=5)

# Прогноз среднего
y_forecasts.mean

In [ ]:
y_forecasts.residual_variance

In [ ]:
# прогноз волатильности
y_forecasts.variance

In [ ]:
am = arch_model(y, mean='ARX', lags=1, vol='EGARCH', p=1, q=1, o=0)

res = am.fit()

y_forecasts = res.forecast(horizon=1)

# Прогноз среднего
y_forecasts.mean

In [ ]:
# прогноз волатильности
y_forecasts.variance

## Подгонка GARCH-in-Mean

Спецификация

\begin{align*}
	y_t&=\alpha_0++\kappa\log \sigma^2_t+\phi y_{t-1}+u_t \\ u_t&=\sigma_t \epsilon_t \\
	\sigma^\lambda_t &=\omega+\alpha |\epsilon_{t-1}|^\lambda+\beta\sigma^\lambda_{t-1}
\end{align*}

где $\lambda=2$

In [ ]:
gim = ARCHInMean(y, lags=1, volatility=GARCH(p=1,o=0, q=1), form='log')
res = gim.fit()

res.summary()